## [Day 13](https://adventofcode.com/2020/day/13)

Hmm so this problem has to do with a number of busses departing and we're asked to find which number bus line comes soonest after the first specified number....

This seems really simple and I'm kinda wondering if I missing anything here... I'm sure the x's will come back to be meaningful in part 2. 

In [1]:
import pandas as pd
import numpy as np
import math

busses = open('../inputs/d13.txt').read().splitlines()
busses

['1004098',
 '23,x,x,x,x,x,x,x,x,x,x,x,x,41,x,x,x,x,x,x,x,x,x,509,x,x,x,x,x,x,x,x,x,x,x,x,13,17,x,x,x,x,x,x,x,x,x,x,x,x,x,x,29,x,401,x,x,x,x,x,37,x,x,x,x,x,x,x,x,x,x,x,x,19']

In [2]:
dept_time = int(busses[0])
busses = busses[1].split(',')
routes = [int(x) for x in busses if x != 'x']
routes[:5]

[23, 41, 509, 13, 17]

So now I'm going to make the departure time negative and take remainders by the routes.

In [3]:
waits = [-1*dept_time % x for x in routes]
waits

[13, 33, 159, 9, 7, 27, 6, 8, 14]

In [4]:
position = waits.index(min(waits))
waits[position]*routes[position]

2406

Hmmm that was about as easy as I expected...

### Part 2

So this just looks like a Chinese remainder theorem problem. The modulo for those  is their position within the original list. We basically need to find the value x such that

$$ x\text{ mod 23 }= 0$$
$$ x\text{ mod 41 }= -13$$

Ect.

In [5]:
# So we'll start with finding those indexes:
remainders = [-1*busses.index(str(x)) for x in routes]
remainders

[0, -13, -23, -36, -37, -52, -54, -60, -73]

What took me a while is to think of this with the negative signs. Instead of how far from when the bus comes, we're looking for how from from time $t$.

I more or less remember this from school so I'll just go ahead and try to solve it using the [steps here](https://brilliant.org/wiki/chinese-remainder-theorem/).

In [6]:
# first thing we do is check that all the divisors are pairwise coprime
# Writing a function for this seems.... tedious so I just visually looked it over
# and can see that we have only primes in the list
routes

[23, 41, 509, 13, 17, 29, 401, 37, 19]

In [7]:
# next we compute the product of all the modulos:
mod_prod = pd.Series(routes).product()

# Now we compute the division of this by each modulo
y_i = [mod_prod//x for x in routes]

# Then the multiplicative inverses of each of these y_i mod route
inverses = []
for i in range(len(routes)):
    for val in range(routes[i]):
        if (val*y_i[i]) % routes[i] == 1:
            inverses.append(val)
            break

In [8]:
# now the solution we can do with vectorized series:
sol = (pd.Series(remainders)*pd.Series(y_i)*pd.Series(inverses)).sum()
sol % mod_prod

225850756401039

In [9]:
def isPrime(n):
 
    # Corner case
    if (n <= 1):
        return False
 
    # Check from 2 to n-1
    for i in range(2, n):
        if (n % i == 0):
            return False
 
    return True

list(map(isPrime, routes))

[True, True, True, True, True, True, True, True, True]

In [10]:
y_i

# 225850756401045 no go

[37704363028163,
 21151228040189,
 1703733496361,
 66707719203673,
 51011785273397,
 29903460332681,
 2162594388149,
 23437847287777,
 45642123665671]